In [136]:
from google.colab import files

a = files.upload()

Saving data.csv to data.csv


In [137]:
import tensorflow as tf
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
from numpy import diff
import random
%matplotlib inline
import shutil
import tensorflow.contrib.learn as tflearn
import tensorflow.contrib.layers as tflayers
from tensorflow.contrib.learn.python.learn import learn_runner
import tensorflow.contrib.metrics as metrics
import tensorflow.contrib.rnn as rnn
from sklearn.model_selection import train_test_split
!git clone https://github.com/EBjerrum/Deep-Chemometrics.git
  
def dataaugment(x, betashift = 0.05, slopeshift = 0.05,multishift = 0.05):
    #Shift of baseline
    #calculate arrays
    beta = np.random.random(size=(x.shape[0],1))*2*betashift-betashift
    slope = np.random.random(size=(x.shape[0],1))*2*slopeshift-slopeshift + 1
    #Calculate relative position
    axis = np.array(range(x.shape[1]))/float(x.shape[1])
    #Calculate offset to be added
    offset = slope*(axis) + beta - axis - slope/2. + 0.5

    #Multiplicative
    multi = np.random.random(size=(x.shape[0],1))*2*multishift-multishift + 1

    x = multi*x + offset

    return x
  
data = pd.read_csv('data.csv')
spectrums= np.array(data.values[:, 0:288],dtype = np.int32)
labels = data.iloc[:, 288].unique()
categories= data.iloc[:, 288].apply(lambda name: np.squeeze(np.where(labels == name))).values
portion = int(4/5 *spectrums.shape[0])
#Split train and test set
x_train, x_test, y_train,y_test = train_test_split(spectrums[:portion],categories[:portion],test_size=0.2,random_state=42)

x_check = spectrums[portion:]
y_check = categories[portion:]

%cd Deep-Chemometrics

from ChemUtils import GlobalStandardScaler

xscaler = GlobalStandardScaler()
x_train = xscaler.fit_transform(x_train) 
x_test = xscaler.transform(x_test) 
x_check = xscaler.transform(x_check)

yscaler = GlobalStandardScaler()
y_train = yscaler.fit_transform(y_train)
y_test = yscaler.transform(y_test)
y_check = yscaler.transform(y_check)

#EMSC rescaling
from ChemUtils import EmscScaler
emsc = EmscScaler()
emsc.fit(x_train)
X_train_emsc = emsc.transform((x_train))
X_test_emsc = emsc.transform(x_test)
X_check_emsc = emsc.transform(x_check)

#Data Augment a single spectrum

#First Spectrum
X = x_test[0:1]
#Repeating the spectrum 10x
X = np.repeat(X, repeats=10, axis=0)
#Augment (Large pertubations for illustration)
X_aug = dataaugment(X,betashift = 0.5, slopeshift = 0.5,multishift = 0.5)
shift = np.std(x_train)*0.1
X_train_aug = np.repeat(X_train_emsc, repeats=1000, axis=0)
X_train_aug = dataaugment(X_train_aug, betashift = shift, slopeshift = 0.05, multishift = shift)
y_train_aug = np.repeat(y_train, repeats=1000, axis=0) #y_train is simply repeated

shift = np.std(x_test)*0.1
X_test_aug = np.repeat(X_test_emsc, repeats=5000, axis=0)
X_test_aug = dataaugment(X_test_aug, betashift = shift, slopeshift = 0.05, multishift = shift)
y_test_aug = np.repeat(y_test, repeats=5000, axis=0) #y_test is simply repeated


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Reshape, GaussianNoise, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau


input_dim = x_train.shape[1]

#The model
def make_model():
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim= input_dim))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(labels.size, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model
  
model = make_model()
y_train_aug = keras.utils.to_categorical(y_train_aug, labels.size)
y_test_aug = keras.utils.to_categorical(y_test_aug, labels.size)

h = model.fit(X_train_aug, y_train_aug, epochs=50, batch_size=1024, validation_data=(X_test_aug, y_test_aug))

# # serialize model to JSON
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# files.download('model.h5') 
# files.download('model.json') 



Cloning into 'Deep-Chemometrics'...
remote: Enumerating objects: 43, done.
remote: Total 43 (delta 0), reused 0 (delta 0), pack-reused 43
Unpacking objects: 100% (43/43), done.
/content/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics/Deep-Chemometrics
Train on 30000 samples, validate on 40000 samples
Epoch 1/50
30000/30000 [==============================] - 1s 46us/step - loss: 0.2031 - acc: 0.9714 - val_loss: 0.0063 - val_acc: 1.0000
Epoch 2/50
30000/30000 [==============================] - 1s 18us/step - loss: 0.0031 - acc: 1.0000 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 3/50
30000/30000 [==============================] - 1s 18us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 6.6386e-04 - val_acc: 1.0000
Epoch 4/50
30000/30000 [==============================] - 1s 18us/step - loss: 6.9509e-04 - acc: 1.0000 - val_loss:

In [144]:
xscaler = GlobalStandardScaler()
x = xscaler.fit_transform(spectrums) 
emsc = EmscScaler()
emsc.fit(x)
d = emsc.transform(xscaler.transform(spectrums[15]).reshape(1, 288))
print(d.shape)
a = model.predict(d)
index = np.argmax(a)
if (index == labels.size - 1):
  index = -1
print(labels[index + 1])

(1, 288)
White
